In [0]:
%sql
/* 03_silver_claims_enriched_join.ipynb

SOURCE: Snapshot join of current claims and current provider attributes.
OUTPUT: kardia_silver.silver_claims_enriched (for Gold KPIs)

PATTERN: LEFT JOIN from claims to SCD-2 providers, keeping only the current provider row.
TRIGGER: CTAS; rerun after any update to claims or providers.

NOTE:
- LEFT JOIN preserves all claims
- is_current = true ensures only the current row is used
*/

-- 1. Ensure database exists
CREATE DATABASE IF NOT EXISTS kardia_silver;

-- 2. Join claims to current provider attributes (no CTE – inline subquery)
CREATE OR REPLACE TABLE kardia_silver.silver_claims_enriched AS
WITH current_providers AS (
    SELECT provider_id, provider_specialty, provider_location
    FROM kardia_silver.silver_providers
    WHERE is_current = true
)
SELECT
    c.claim_id,
    c.patient_id,
    c.provider_id,
    c.claim_amount,
    c.claim_date,
    c.diagnosis_code,
    c.procedure_code,
    c.claim_status,
    c.claim_type,
    c.claim_submission_method,
    c._ingest_ts,
    p.provider_specialty,
    p.provider_location
FROM kardia_silver.silver_claims c
LEFT JOIN current_providers p
ON c.provider_id = p.provider_id;